In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import scipy
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

Y_train = mnist.train.labels
Y_test  = mnist.test.labels

def one_hot_encode(t):
    R = np.zeros((len(t), np.max(t)+1))
    R[np.arange(len(t)), t] = 1.0
    return R

def one_hot_decode(R):
    return np.argmax(R, axis=1)


def softmax(y):
    "for each row in y, denoted as y_row, compute normalized(exp(y_row))"
    y = np.exp(y)
    return y / np.sum(y, axis=1).reshape(-1,1)

def nnet(X, W):
    return softmax(X.dot(W))

Y_train2 = one_hot_encode(Y_train)
Y_test2  = one_hot_encode(Y_test)
np.random.seed(123)
X_train = np.insert(mnist.train.images, 0, 1.0, axis=1) # add a column of ones
X_test  = np.insert(mnist.test.images, 0, 1.0, axis=1)  # add a column of ones
W = np.zeros(shape=(785, 10))


np.mean(one_hot_decode(nnet(X_test, W)) == Y_test)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


0.098000000000000004

In [55]:
def print_var(var):
    def namestr(obj, namespace):
        return [name for name in namespace if namespace[name] is obj]
    print("{:<20} is {}".format(str(namestr(var,globals())), var))


In [56]:
x = tf.placeholder(tf.float32, [None,785])
y = tf.placeholder(tf.float32, [None, 10])
W = tf.Variable(tf.zeros([785,10]))
#x,y,W
y_pred = tf.nn.softmax(tf.matmul(x,W))
print_var(y_pred)
cross_entropy = -tf.reduce_sum(y * tf.log(y_pred))/tf.cast(tf.shape(y), tf.float32)
print_var(cross_entropy)
accuracy = tf.reduce_mean(
    tf.cast(tf.equal(
        tf.argmax(y_pred,1), tf.argmax(y,1)
        )
        , tf.float32)
    )
print(accuracy)
#for i in dir(y_pred):
#    print(i)

['y_pred']           is Tensor("Softmax_28:0", shape=(?, 10), dtype=float32)
['cross_entropy']    is Tensor("truediv_14:0", shape=(2,), dtype=float32)
Tensor("Mean_7:0", shape=(), dtype=float32)


In [46]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
print_var(train_step)

['train_step'] is name: "GradientDescent_1"
op: "NoOp"
input: "^GradientDescent_1/update_Variable_25/ApplyGradientDescent"



In [57]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [58]:
# W = np.random.uniform(size=(785,10)) # Random initialize
# eta = 0.5 # Learning rate
# cnt = 0
# splitnum = 100
# s = X_train.shape[0]//splitnum
# maxiter = 100
# while True:
#     idx = np.random.permutation(np.arange(X_train.shape[0]))
#     for j in range(splitnum):
#         start = j*s
#         stop = (j+1)*s
#         sample = idx[j*s:(j+1)*s]
#         #print("Interval:",j, j*s, (j+1)*s)
#         this_sub_X = X_train[sample,:]
#         this_sub_Y = Y_train2[sample]
#         sess.run(
            
#             )
#         #grad = cross_entropy_gradient(W, this_sub_X, this_sub_Y)
#         #W = W - eta*grad
#     #grad = cross_entropy_gradient(W, X_train, Y_train2)

#     cnt+=1

    
#     if np.sum(grad**2) < 1e-3: break
    
# #     for i in enumurate(cnt):
# #         W_old = W

#     print("{:4d} cross_entropy {:.4f} accuracy {:.4f} accuracy_test {:4f} grad {:.4f}".format(
#         cnt, cross_entropy(W,X_train, Y_train2), accuracy(W,X_train,Y_train),accuracy(W,X_test,Y_test),np.sum(grad**2)
#     ),end="\n")
#     print(np.max(W),np.min(W))
#     continue

In [4]:
import tensorflow as tf
x = tf.placeholder(tf.float32, [None, 785])
y = tf.placeholder(tf.float32, [None, 10])
W = tf.Variable(tf.zeros([785, 10]))
ypred = tf.nn.softmax(tf.matmul(x, W))
cross_entropy = -tf.reduce_sum(y * tf.log(ypred))/tf.cast(tf.shape(y), tf.float32)[0]
accuracy = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(ypred, 1), tf.argmax(y, 1)), tf.float32)
)
#train_step = W-eta*cross_etropy_grad
eta = 0.5
train_step = tf.train.GradientDescentOptimizer(eta).minimize(cross_entropy)

In [5]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

splitnum = 100
s = X_train.shape[0]//splitnum # batch size
maxiter = 100
for i in range(maxiter):
    idx = np.random.permutation(np.arange(X_train.shape[0]))
    for j in range(splitnum):
        sample = idx[j*s:(j+1)*s]
        #grad = cross_entropy_grad(W, X_train[sample,:], Y_train2[sample])
        #W = W - eta*grad
        sess.run(train_step,
                feed_dict={
                        x: X_train[sample,:],
                        y: Y_train2[sample,:]
                }
                )
    
    ac = sess.run(accuracy, feed_dict={x: X_train, y: Y_train2})
    print("%4d: accuracy=%.6f" %
         (i+1, ac))

   1: accuracy=0.892564
   2: accuracy=0.902218
   3: accuracy=0.908509
   4: accuracy=0.911527
   5: accuracy=0.914091
   6: accuracy=0.916218
   7: accuracy=0.916891
   8: accuracy=0.919145
   9: accuracy=0.919709
  10: accuracy=0.920418
  11: accuracy=0.921127
  12: accuracy=0.922018
  13: accuracy=0.923073
  14: accuracy=0.922836
  15: accuracy=0.921745
  16: accuracy=0.923091
  17: accuracy=0.923218
  18: accuracy=0.924873
  19: accuracy=0.925127
  20: accuracy=0.924855
  21: accuracy=0.925473
  22: accuracy=0.925873
  23: accuracy=0.926309
  24: accuracy=0.926491
  25: accuracy=0.926691
  26: accuracy=0.926945
  27: accuracy=0.926982
  28: accuracy=0.927327
  29: accuracy=0.927073
  30: accuracy=0.927291
  31: accuracy=0.927491
  32: accuracy=0.927418
  33: accuracy=0.928127
  34: accuracy=0.927745
  35: accuracy=0.927982
  36: accuracy=0.928418
  37: accuracy=0.928691
  38: accuracy=0.928636
  39: accuracy=0.929291
  40: accuracy=0.928873
  41: accuracy=0.929273
  42: accuracy=0

In [4]:
def accuracy(W, X, Y):
    return np.mean(one_hot_decode(nnet(X,W)) == Y)

In [5]:
accuracy(W,X_train,Y_train)

0.098981818181818185

In [9]:
def cross_entropy(W, X, Y):
    Y_pred = nnet(X,W)
    return -np.sum(Y * np.log(Y_pred))/X.shape[0]

In [11]:
cross_entropy(W,X_train,Y_train2)

2.3025850929940432

In [16]:
error = lambda w: cross_entropy(w.reshape(785, 10),X_train,Y_train2)

#res = scipy.optimize.minimize(error,W.ravel(), method='Nelder-Mead')

In [31]:
%%script = False
# BROKEN
def cross_entropy_gradient(W,X,Y):
    n = X.shape[0]
    nnet_out = nnet(W,X)
    this_minus =  Y - nnet_out
    return -X.T @ this_minus
    

Couldn't find program: '='


In [28]:
def cross_entropy_gradient(W,X,Y):
    return -(X.T @ (Y-nnet(X,W)))/X.shape[0]

In [29]:
cross_entropy_gradient(W, X_train, Y_train2)

array([[ 0.00101818, -0.01234545,  0.00054545, ..., -0.00390909,
         0.00201818,  0.00083636],
       [-0.        , -0.        , -0.        , ..., -0.        ,
        -0.        , -0.        ],
       [-0.        , -0.        , -0.        , ..., -0.        ,
        -0.        , -0.        ],
       ..., 
       [-0.        , -0.        , -0.        , ..., -0.        ,
        -0.        , -0.        ],
       [-0.        , -0.        , -0.        , ..., -0.        ,
        -0.        , -0.        ],
       [-0.        , -0.        , -0.        , ..., -0.        ,
        -0.        , -0.        ]])

In [40]:
W = np.zeros(shape=(785,10)) # Starting point
eta = 0.5 # Learning rate
cnt = 0
while True:
    
    #print(cnt,end="|")
    print("{:4d} cross_entropy {:.6f} accuracy {:.6f} accuracy_test {:6f} ".format(
        cnt, cross_entropy(W,X_train, Y_train2), accuracy(W,X_train,Y_train),accuracy(W,X_test,Y_test)),end="\n")
    cnt+=1
    W_old = W
    grad = cross_entropy_gradient(W_old, X_train, Y_train2)
    W = W - eta*grad
    
    if np.sum(grad**2) < 1e-4: break
    

   0 cross_entropy 2.302585 accuracy 0.098982 accuracy_test 0.098000 
   1 cross_entropy 1.832930 accuracy 0.662927 accuracy_test 0.670500 
   2 cross_entropy 1.515830 accuracy 0.777473 accuracy_test 0.790200 
   3 cross_entropy 1.303822 accuracy 0.772745 accuracy_test 0.784000 
   4 cross_entropy 1.157700 accuracy 0.805764 accuracy_test 0.815600 
   5 cross_entropy 1.051340 accuracy 0.805400 accuracy_test 0.817200 
   6 cross_entropy 0.972172 accuracy 0.820200 accuracy_test 0.829700 
   7 cross_entropy 0.909678 accuracy 0.823127 accuracy_test 0.832500 
   8 cross_entropy 0.860149 accuracy 0.830564 accuracy_test 0.838700 
   9 cross_entropy 0.818405 accuracy 0.833218 accuracy_test 0.843400 
  10 cross_entropy 0.783892 accuracy 0.838582 accuracy_test 0.847100 
  11 cross_entropy 0.753953 accuracy 0.841636 accuracy_test 0.851700 
  12 cross_entropy 0.728395 accuracy 0.844909 accuracy_test 0.855300 
  13 cross_entropy 0.705970 accuracy 0.848327 accuracy_test 0.858900 
  14 cross_entropy 0

 118 cross_entropy 0.382064 accuracy 0.896691 accuracy_test 0.904200 
 119 cross_entropy 0.381455 accuracy 0.896818 accuracy_test 0.904300 
 120 cross_entropy 0.380854 accuracy 0.897036 accuracy_test 0.904300 
 121 cross_entropy 0.380260 accuracy 0.897091 accuracy_test 0.904500 
 122 cross_entropy 0.379674 accuracy 0.897164 accuracy_test 0.904800 
 123 cross_entropy 0.379094 accuracy 0.897236 accuracy_test 0.905000 
 124 cross_entropy 0.378522 accuracy 0.897418 accuracy_test 0.905200 
 125 cross_entropy 0.377957 accuracy 0.897473 accuracy_test 0.905200 
 126 cross_entropy 0.377399 accuracy 0.897655 accuracy_test 0.905200 
 127 cross_entropy 0.376847 accuracy 0.897745 accuracy_test 0.905000 
 128 cross_entropy 0.376302 accuracy 0.897764 accuracy_test 0.905000 
 129 cross_entropy 0.375763 accuracy 0.897691 accuracy_test 0.905000 
 130 cross_entropy 0.375230 accuracy 0.897855 accuracy_test 0.905100 
 131 cross_entropy 0.374704 accuracy 0.897982 accuracy_test 0.905400 
 132 cross_entropy 0

 236 cross_entropy 0.339688 accuracy 0.905836 accuracy_test 0.912400 
 237 cross_entropy 0.339470 accuracy 0.905873 accuracy_test 0.912400 
 238 cross_entropy 0.339254 accuracy 0.905964 accuracy_test 0.912400 
 239 cross_entropy 0.339040 accuracy 0.906055 accuracy_test 0.912600 
 240 cross_entropy 0.338826 accuracy 0.906200 accuracy_test 0.912600 
 241 cross_entropy 0.338614 accuracy 0.906218 accuracy_test 0.912600 
 242 cross_entropy 0.338404 accuracy 0.906345 accuracy_test 0.912600 
 243 cross_entropy 0.338194 accuracy 0.906436 accuracy_test 0.912600 
 244 cross_entropy 0.337986 accuracy 0.906455 accuracy_test 0.912600 
 245 cross_entropy 0.337779 accuracy 0.906509 accuracy_test 0.912700 
 246 cross_entropy 0.337573 accuracy 0.906545 accuracy_test 0.912700 
 247 cross_entropy 0.337369 accuracy 0.906564 accuracy_test 0.912700 
 248 cross_entropy 0.337165 accuracy 0.906618 accuracy_test 0.912700 
 249 cross_entropy 0.336963 accuracy 0.906655 accuracy_test 0.912800 
 250 cross_entropy 0

 354 cross_entropy 0.320675 accuracy 0.911164 accuracy_test 0.915900 
 355 cross_entropy 0.320555 accuracy 0.911200 accuracy_test 0.916000 
 356 cross_entropy 0.320435 accuracy 0.911255 accuracy_test 0.916200 
 357 cross_entropy 0.320316 accuracy 0.911291 accuracy_test 0.916200 
 358 cross_entropy 0.320197 accuracy 0.911291 accuracy_test 0.916200 
 359 cross_entropy 0.320079 accuracy 0.911291 accuracy_test 0.916200 
 360 cross_entropy 0.319961 accuracy 0.911345 accuracy_test 0.916200 
 361 cross_entropy 0.319844 accuracy 0.911400 accuracy_test 0.916300 
 362 cross_entropy 0.319727 accuracy 0.911491 accuracy_test 0.916400 
 363 cross_entropy 0.319611 accuracy 0.911527 accuracy_test 0.916500 
 364 cross_entropy 0.319495 accuracy 0.911545 accuracy_test 0.916500 
 365 cross_entropy 0.319379 accuracy 0.911545 accuracy_test 0.916500 
 366 cross_entropy 0.319264 accuracy 0.911545 accuracy_test 0.916500 
 367 cross_entropy 0.319150 accuracy 0.911582 accuracy_test 0.916500 
 368 cross_entropy 0

 472 cross_entropy 0.309173 accuracy 0.914564 accuracy_test 0.918500 
 473 cross_entropy 0.309093 accuracy 0.914618 accuracy_test 0.918500 
 474 cross_entropy 0.309014 accuracy 0.914655 accuracy_test 0.918500 
 475 cross_entropy 0.308936 accuracy 0.914691 accuracy_test 0.918500 
 476 cross_entropy 0.308857 accuracy 0.914727 accuracy_test 0.918500 
 477 cross_entropy 0.308779 accuracy 0.914800 accuracy_test 0.918500 
 478 cross_entropy 0.308701 accuracy 0.914745 accuracy_test 0.918500 
 479 cross_entropy 0.308623 accuracy 0.914782 accuracy_test 0.918500 
 480 cross_entropy 0.308546 accuracy 0.914782 accuracy_test 0.918500 
 481 cross_entropy 0.308468 accuracy 0.914800 accuracy_test 0.918400 
 482 cross_entropy 0.308391 accuracy 0.914782 accuracy_test 0.918400 
 483 cross_entropy 0.308314 accuracy 0.914800 accuracy_test 0.918400 
 484 cross_entropy 0.308238 accuracy 0.914818 accuracy_test 0.918400 
 485 cross_entropy 0.308161 accuracy 0.914855 accuracy_test 0.918600 
 486 cross_entropy 0

 590 cross_entropy 0.301217 accuracy 0.916455 accuracy_test 0.919000 
 591 cross_entropy 0.301160 accuracy 0.916527 accuracy_test 0.919100 
 592 cross_entropy 0.301102 accuracy 0.916527 accuracy_test 0.919100 
 593 cross_entropy 0.301045 accuracy 0.916545 accuracy_test 0.919100 
 594 cross_entropy 0.300988 accuracy 0.916582 accuracy_test 0.919100 
 595 cross_entropy 0.300931 accuracy 0.916600 accuracy_test 0.919100 
 596 cross_entropy 0.300875 accuracy 0.916600 accuracy_test 0.919200 
 597 cross_entropy 0.300818 accuracy 0.916636 accuracy_test 0.919200 
 598 cross_entropy 0.300762 accuracy 0.916636 accuracy_test 0.919200 
 599 cross_entropy 0.300705 accuracy 0.916655 accuracy_test 0.919200 
 600 cross_entropy 0.300649 accuracy 0.916673 accuracy_test 0.919200 
 601 cross_entropy 0.300593 accuracy 0.916691 accuracy_test 0.919200 
 602 cross_entropy 0.300537 accuracy 0.916691 accuracy_test 0.919200 
 603 cross_entropy 0.300481 accuracy 0.916709 accuracy_test 0.919200 
 604 cross_entropy 0

In [49]:
from sklearn.utils import shuffle
#X = np.array([[1., 0.], [2., 1.], [0., 0.]])
#y = np.array([0, 1, 2])
#X, y = shuffle(X, y, random_state=0)


W = np.zeros(shape=(785,10)) # Starting point
eta = 0.5 # Learning rate
cnt = 0
splitnum = 100
while True:
    cnt+=1
    
    Xshuf, Yshuf = shuffle(X_train, Y_train2, random_state=0)    
    
    #training, test = 
    #print(len(X_train))
    #print()
    stepsize = Xshuf.shape[0]/splitnum
    np.arange(0,stepsize)
    print(_)
    break
    
    if np.sum(grad**2) < 1e-4: break
    
    for i in enumurate(cnt):
        W_old = W
        grad = cross_entropy_gradient(W_old, X_train, Y_train2)
        W = W - eta*grad

    print("{:4d} cross_entropy {:.6f} accuracy {:.6f} accuracy_test {:6f} ".format(
        cnt, cross_entropy(W,X_train, Y_train2), accuracy(W,X_train,Y_train),accuracy(W,X_test,Y_test)),end="\n")
    continue
    
    

[[ 0.00101818 -0.01234545  0.00054545 ..., -0.00390909  0.00201818
   0.00083636]
 [-0.         -0.         -0.         ..., -0.         -0.         -0.        ]
 [-0.         -0.         -0.         ..., -0.         -0.         -0.        ]
 ..., 
 [-0.         -0.         -0.         ..., -0.         -0.         -0.        ]
 [-0.         -0.         -0.         ..., -0.         -0.         -0.        ]
 [-0.         -0.         -0.         ..., -0.         -0.         -0.        ]]


## Teacher solution

In [84]:
W = np.zeros(shape=(785,10)) # Starting point
W = np.random.uniform(size=(785,10)) # Random initialize
#raise
eta = 0.5 # Learning rate
cnt = 0
splitnum = 100
s = X_train.shape[0]//splitnum
while True:
    idx = np.random.permutation(np.arange(X_train.shape[0]))
    #print(idx.shape)
    #print(idx)
    for j in range(splitnum):
        start = j*s
        stop = (j+1)*s
        sample = idx[j*s:(j+1)*s]
        #print("Interval:",j, j*s, (j+1)*s)
        this_sub_X = X_train[sample,:]
        this_sub_Y = Y_train2[sample]
        grad = cross_entropy_gradient(W, this_sub_X, this_sub_Y)
        W = W - eta*grad
    grad = cross_entropy_gradient(W, X_train, Y_train2)
        
        #print(sample)
        #break
    cnt+=1
    #break
    #Xshuf, Yshuf = shuffle(X_train, Y_train2, random_state=0)    
    
    #training, test = 
    #print(len(X_train))
    #print()
    #stepsize = Xshuf.shape[0]/splitnum
    #np.arange(0,stepsize)
    #print(_)
    #break
    
    if np.sum(grad**2) < 1e-3: break
    
#     for i in enumurate(cnt):
#         W_old = W

    print("{:4d} cross_entropy {:.4f} accuracy {:.4f} accuracy_test {:4f} grad {:.4f}".format(
        cnt, cross_entropy(W,X_train, Y_train2), accuracy(W,X_train,Y_train),accuracy(W,X_test,Y_test),np.sum(grad**2)
    ),end="\n")
    print(np.max(W),np.min(W))
    continue

   1 cross_entropy 0.4738 accuracy 0.8586 accuracy_test 0.865500 grad 0.0086
1.35086815491 -0.226395603667
   2 cross_entropy 0.4006 accuracy 0.8818 accuracy_test 0.889100 grad 0.0060
1.37478443396 -0.32171679567
   3 cross_entropy 0.3684 accuracy 0.8925 accuracy_test 0.898600 grad 0.0054
1.38363826266 -0.468486214251
   4 cross_entropy 0.3480 accuracy 0.9001 accuracy_test 0.904900 grad 0.0017
1.50694803339 -0.591020944865


In [87]:
best_acc_train = 0
best_acc_test = 0
for i in range(100):
    W = np.random.uniform(-2,2,size=(785,10)) # Random initialize
    acc_train = accuracy(W,X_train,Y_train)
    #acc_test = accuracy(W,X_test,Y_test)
    #break
    if acc_train > best_acc_train:
        best_acc_train = acc_train
        #best_acc_test = acc_test
    print(acc_train,best_acc_test)
print(best_acc_train,best_acc_test)
    


0.151618181818 0
0.114236363636 0
0.113145454545 0
0.165654545455 0
0.0910545454545 0
0.0705272727273 0
0.0955272727273 0
0.0921636363636 0
0.1108 0
0.104436363636 0
0.0608727272727 0
0.0820363636364 0
0.114436363636 0
0.103254545455 0
0.081 0
0.0625272727273 0
0.0974909090909 0
0.100545454545 0
0.1142 0
0.120163636364 0
0.0842545454545 0
0.0779272727273 0
0.0836 0
0.0797818181818 0
0.0933818181818 0
0.0609454545455 0
0.0972727272727 0
0.116327272727 0
0.104818181818 0
0.129527272727 0
0.0733454545455 0
0.0789818181818 0
0.1126 0
0.0769454545455 0
0.0962363636364 0
0.103127272727 0
0.0942909090909 0
0.102963636364 0
0.0943454545455 0
0.116618181818 0
0.0549818181818 0
0.0905272727273 0
0.0607454545455 0
0.100672727273 0
0.096 0
0.103236363636 0
0.0964 0
0.0559090909091 0
0.135472727273 0
0.140836363636 0
0.0790909090909 0
0.0914727272727 0
0.0772909090909 0
0.136709090909 0
0.109472727273 0
0.138909090909 0
0.106654545455 0
0.0851454545455 0
0.148945454545 0
0.0784727272727 0
0.1113636

KeyboardInterrupt: 